In [6]:
# import os
# import requests
# import pandas as pd
# from concurrent.futures import ThreadPoolExecutor, as_completed

# def obter_autenticacao():
#     username = os.getenv("ELOQUA_USERNAME")
#     password = os.getenv("ELOQUA_PASSWORD")
#     return (username, password)

# def obter_landing_pages(microsite_id, page=1):
#     url = f"https://secure.p04.eloqua.com/API/REST/2.0/assets/landingPages?search=micrositeId={microsite_id}&page={page}"
#     try:
#         response = requests.get(url, auth=obter_autenticacao())
#         response.raise_for_status()
#         return response.json()
#     except requests.exceptions.RequestException as e:
#         print(f"Erro ao obter landing pages da página {page}: {e}")
#         return {"elements": []}

# def obter_landing_page_detalhes(landing_page_id):
#     url = f"https://secure.p04.eloqua.com/api/rest/2.0/assets/landingPage/{landing_page_id}"
#     try:
#         response = requests.get(url, auth=obter_autenticacao())
#         response.raise_for_status()
#         return response.json()
#     except requests.exceptions.RequestException as e:
#         print(f"Erro ao obter detalhes da landing page {landing_page_id}: {e}")
#         return {}

# def processar_landing_page(landing_page):
#     landing_page_detalhes = obter_landing_page_detalhes(landing_page["id"])
#     forms = landing_page_detalhes.get("forms", [])

#     # Verifica se a chave "forms" existe e se não está vazia
#     if forms:
#         tem_botao = False

#         for form in forms:
#             for element in form.get("elements", []):
#                 if element.get("altText") == "Submit":
#                     return {
#                         "Landing Page ID": landing_page["id"],
#                         "Landing Page Name": landing_page["name"],
#                         "Form ID": form["id"],
#                         "Form Name": form["name"],
#                         "Button": "Y",
#                     }

#         if not tem_botao:
#             return {
#                 "Landing Page ID": landing_page["id"],
#                 "Landing Page Name": landing_page["name"],
#                 "Form ID": "",
#                 "Form Name": "",
#                 "Button": "N"
#             }
#     else:
#         return None  # Retorna None se não houver "forms"

# def encontrar_botao_register_now(microsite_id):
#     landing_pages_data = []
#     landing_pages = obter_landing_pages(microsite_id)
#     total = landing_pages["total"]
#     pageSize = landing_pages["pageSize"]
#     num_pages = (total // pageSize) + (1 if total % pageSize else 0)

#     with ThreadPoolExecutor() as executor:
#         futures = [executor.submit(processar_landing_page, landing_page) 
#                    for page in range(1, num_pages + 1)
#                    for landing_page in obter_landing_pages(microsite_id, page)["elements"]]
        
#         for future in as_completed(futures):
#             resultado = future.result()
#             # Só adiciona à lista se o resultado não for None
#             if resultado is not None:
#                 landing_pages_data.append(resultado)

#     return landing_pages_data

# def main():
#     microsite_id = 10
#     dados_botao_register_now = encontrar_botao_register_now(microsite_id)

#     if dados_botao_register_now:
#         df = pd.DataFrame(dados_botao_register_now)
#         df.to_excel(f"landing_pages_data_{microsite_id}.xlsx", index=False)
#         print("Dados salvos em landing_pages_data.xlsx")
#     else:
#         print("Nenhum botão ou classe encontrada nas landing pages.")

# if __name__ == "__main__":
#     main()

Dados salvos em landing_pages_data.xlsx


In [26]:
import os
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

def obter_autenticacao():
    username = os.getenv("ELOQUA_USERNAME")
    password = os.getenv("ELOQUA_PASSWORD")
    return (username, password)

def obter_landing_pages(microsite_id, page=1):
    url = f"https://secure.p04.eloqua.com/API/REST/2.0/assets/landingPages?search=micrositeId={microsite_id}&page={page}"
    try:
        response = requests.get(url, auth=obter_autenticacao())
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter landing pages da página {page}: {e}")
        return {"elements": []}

def obter_landing_page_detalhes(landing_page_id):
    url = f"https://secure.p04.eloqua.com/api/rest/2.0/assets/landingPage/{landing_page_id}"
    try:
        response = requests.get(url, auth=obter_autenticacao())
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter detalhes da landing page {landing_page_id}: {e}")
        return {}

def processar_landing_page(landing_page):
    landing_page_detalhes = obter_landing_page_detalhes(landing_page["id"])
    forms = landing_page_detalhes.get("forms", [])
    
    if forms:
        for form in forms:
            html = form.get("html", "")
            # Verifica input type="image"
            if 'input type="image"' in html:
                return {
                    "Landing Page ID": landing_page["id"],
                    "Landing Page Name": landing_page["name"],
                    "Form ID": form["id"],
                    "Form Name": form["name"],
                    "Button": "Y (image)",
                }
        # Se não encontrar nenhum dos botões, retorna "N"
        return {
            "Landing Page ID": landing_page["id"],
            "Landing Page Name": landing_page["name"],
            "Form ID": "",
            "Form Name": "",
            "Button": "N",
        }
    else:
        return None

def encontrar_botao_register_now(microsite_id):
    landing_pages_data = []
    landing_pages = obter_landing_pages(microsite_id)
    total = landing_pages.get("total", 0)
    pageSize = landing_pages.get("pageSize", 1)
    num_pages = (total // pageSize) + (1 if total % pageSize else 0)

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(processar_landing_page, landing_page) 
                   for page in range(1, num_pages + 1)
                   for landing_page in obter_landing_pages(microsite_id, page)["elements"]]
        
        for future in as_completed(futures):
            resultado = future.result()
            if resultado is not None:
                landing_pages_data.append(resultado)

    return landing_pages_data

def main():
    microsite_id = 9
    dados_botao_register_now = encontrar_botao_register_now(microsite_id)

    if dados_botao_register_now:
        df = pd.DataFrame(dados_botao_register_now)
        df.to_excel(f"landing_pages_microsite_{microsite_id}.xlsx", index=False)
        print(f"Dados salvos em landing_pages_data_{microsite_id}.xlsx")
    else:
        print("Nenhum botão ou input de imagem encontrado nas landing pages.")

if __name__ == "__main__":
    main()

Dados salvos em landing_pages_data_9.xlsx


In [20]:
import os
import requests
import pandas as pd

def obter_autenticacao():
    username = os.getenv("ELOQUA_USERNAME")
    password = os.getenv("ELOQUA_PASSWORD")
    return (username, password)

def obter_landing_page_detalhes(landing_page_id):
    url = f"https://secure.p04.eloqua.com/api/rest/2.0/assets/landingPage/{landing_page_id}"
    try:
        response = requests.get(url, auth=obter_autenticacao())
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter detalhes da landing page {landing_page_id}: {e}")
        return {}

def verificar_input_image(json_data):
    if 'forms' in json_data:
        for form in json_data['forms']:
            if 'html' in form:
                # Usando find() para buscar 'input type="image"' no HTML, 
                # ignorando formatação e espaços extras
                if 'input type="image"' in form['html']:
                    print(f"Input type=\"image\" found in form ID: {form['id']}")
                    return True
    return False

# Example usage
landing_page_id = "5202"  # Substitua pelo ID real da landing page
detalhes = obter_landing_page_detalhes(landing_page_id)
if verificar_input_image(detalhes):
    print(r"Input type=\"image\" exists.")
else:
    print(r"Input type=\"image\" does not exist.")

Input type="image" found in form ID: 6043
Input type=\"image\" exists.
